NOTE: watch the flags to choose model_typ between base_model and ft_model!

In [1]:
import jsonlines 

responses= []
#with jsonlines.open('data/base_model_responses.jsonl', mode='r') as reader:
with jsonlines.open('data/ft_model_responses.jsonl', mode='r') as reader:
    for item in reader:
        responses.append(item)

In [20]:
len(responses)

10

In [22]:
responses[9]

{'id': '2092',
 'sentence': "Mongolia 's state copyright official , Gundegma Jargalshaihan , said apologetically that he had just arrived from Ulan Bator and was not aware of the details of the digital agenda .",
 'entities': {'LOC': ['Mongolia', 'Ulan Bator'],
  'MISC': [],
  'ORG': [],
  'PER': ['Gundegma Jargalshaihan']},
 'ft_model': {'response': "\n{'LOC': ['Ulan Bator'], 'MISC': ['Mongolia'], 'ORG': [], 'PER': ['Gundegma Jargalshaihan']}",
  'response_dict': {'LOC': ['Ulan Bator'],
   'MISC': ['Mongolia'],
   'ORG': [],
   'PER': ['Gundegma Jargalshaihan']}}}

In [4]:
def precision(actual, predicted):

    actual = list(set(actual))
    predicted = list(set(predicted))

    actual_lower = [word.lower() for word in actual]
    predicted_lower = [word.lower() for word in predicted]

    if not actual_lower and not predicted_lower:
        return 1.0  # Both lists are empty, so precision is 1 (correct prediction)

    true_positives = sum(1 for p in predicted_lower if p in actual_lower)
    predicted_positives = len(predicted_lower)
    if predicted_positives == 0:
        return 0  # Handle case where there are no predicted positives to avoid division by zero
    return true_positives / predicted_positives

In [5]:
def recall(actual, predicted):
    # remove duplicates
    actual = list(set(actual))
    predicted = list(set(predicted))

    #lower case
    actual_lower = [word.lower() for word in actual]
    predicted_lower = [word.lower() for word in predicted]

    if not actual_lower and not predicted_lower:
        return 1.0  # Both lists are empty, so recall is 1 (correct prediction)

    true_positives = sum(1 for p in predicted_lower if p in actual_lower)
    actual_positives = len(actual_lower)
    if actual_positives == 0:
        return 0  # Handle case where there are no actual positives to avoid division by zero
    return true_positives / actual_positives


In [6]:
def f1_score(actual, predicted):
    prec = precision(actual, predicted)
    rec = recall(actual, predicted)
    if prec + rec == 0:
        return 0  # Handle case where precision + recall is zero to avoid division by zero
    return 2 * (prec * rec) / (prec + rec)

https://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/

The Language-Independent Named Entity Recognition task introduced at CoNLL-2003 measures the performance of the systems in terms of precision, recall and f1-score, where:

“precision is the percentage of named entities found by the learning system that are correct. Recall is the percentage of named entities in the corpus found by the system. A named entity is correct only if it is an exact match of the corresponding entity in the data file.”



In [41]:
# Test case 1: Both lists are empty (correct prediction)
actual1 = []
predicted1 = []
# Test case 2: Both lists contain the same words in the same order (perfect prediction)
actual2 = ["apple", "banana", "cherry"]
predicted2 = ["apple", "banana", "cherry"]
# Test case 3: Both lists contain the same words in different order (perfect prediction)
actual3 = ["apple", "banana", "cherry"]
predicted3 = ["banana", "apple", "cherry"]
# Test case 4: Both lists contain the same words in different case (perfect prediction due to case insensitivity)
actual4 = ["apple", "banana", "cherry"]
predicted4 = ["APPLE", "Banana", "CHERRY"]
# Test case 5: No predicted words match any actual words (zero precision)
actual5 = ["apple", "banana", "cherry"]
predicted5 = ["orange", "grape", "pear"]
# Test case 6: No actual words match any predicted words (zero recall)
actual6 = ["apple", "banana", "cherry"]
predicted6 = ["orange", "grape", "pear"]
# Test case 7: Both lists are large, contain various words, with some correct predictions (non-zero precision and recall)
actual7 = ["apple", "banana", "cherry"] * 1000
predicted7 = ["apple", "pear", "grape"] * 1000
# Test case 8: Both lists are large, contain various words, with no correct predictions (zero precision and recall)
actual8 = ["apple", "banana", "cherry"] * 1000
predicted8 = ["orange", "melon", "strawberry"] * 1000
# Test case 9: Actual list is empty but predicted list is not
actual9 = []
predicted9 = ["apple", "banana", "cherry"]

# Test case 10: Predicted list is empty but actual list is not
actual10 = ["apple", "banana", "cherry"]
predicted10 = []

# Handling Duplicates
# Test case 11: Duplicate elements in actual list
actual11 = ["apple", "banana", "cherry", "apple"]
predicted11 = ["apple", "banana", "cherry"]

# Test case 12: Duplicate elements in predicted list
actual12 = ["apple", "banana", "cherry"]
predicted12 = ["apple", "banana", "banana", "cherry"]


# Calculate metrics for each test case
test_cases = [
    (actual1, predicted1),
    (actual2, predicted2),
    (actual3, predicted3),
    (actual4, predicted4),
    (actual5, predicted5),
    (actual6, predicted6),
    (actual7, predicted7),
    (actual8, predicted8),
    (actual9, predicted9),
    (actual10, predicted10),
    (actual11, predicted11),
    (actual12, predicted12),
]

for i, (actual, predicted) in enumerate(test_cases):
    prec = precision(actual, predicted)
    rec = recall(actual, predicted)
    f1 = f1_score(actual, predicted)
    print(f"Test case {i+1}:")
    print(f"Actual: {actual}")
    print(f"Predicted: {predicted}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1 Score: {f1}")
    print()


Test case 1:
Actual: []
Predicted: []
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Test case 2:
Actual: ['apple', 'banana', 'cherry']
Predicted: ['apple', 'banana', 'cherry']
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Test case 3:
Actual: ['apple', 'banana', 'cherry']
Predicted: ['banana', 'apple', 'cherry']
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Test case 4:
Actual: ['apple', 'banana', 'cherry']
Predicted: ['APPLE', 'Banana', 'CHERRY']
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

Test case 5:
Actual: ['apple', 'banana', 'cherry']
Predicted: ['orange', 'grape', 'pear']
Precision: 0.0
Recall: 0.0
F1 Score: 0

Test case 6:
Actual: ['apple', 'banana', 'cherry']
Predicted: ['orange', 'grape', 'pear']
Precision: 0.0
Recall: 0.0
F1 Score: 0

Test case 7:
Actual: ['apple', 'banana', 'cherry', 'apple', 'banana', 'cherry', 'apple', 'banana', 'cherry', 'apple', 'banana', 'cherry', 'apple', 'banana', 'cherry', 'apple', 'banana', 'cherry', 'apple', 'banana', 'cherry', 'apple', 'banana', 'cherry', 'app

In [23]:
model_type = 'ft_model' #'base_model'

for idx in range(len(responses)):
    scores = {}
    for key in ['PER','ORG','LOC', 'MISC']:
        prec = precision(responses[idx]['entities'][key], responses[idx][model_type]['response_dict'][key])
        rec = recall(responses[idx]['entities'][key], responses[idx][model_type]['response_dict'][key])
        f1  = f1_score(responses[idx]['entities'][key], responses[idx][model_type]['response_dict'][key])
        
        scores[key] = (prec, rec, f1)
    responses[idx][model_type]['scores'] = scores


In [24]:
len(responses)

10

In [25]:
responses[8]

{'id': '2771',
 'sentence': 'SOCCER - LEADING SCOTTISH PREMIER DIVISION SCORERS .',
 'entities': {'LOC': [],
  'MISC': ['SCOTTISH PREMIER DIVISION'],
  'ORG': [],
  'PER': []},
 'ft_model': {'response': "\n{'LOC': ['SCOTTISH'], 'MISC': [], 'ORG': [], 'PER': []}",
  'response_dict': {'LOC': ['SCOTTISH'], 'MISC': [], 'ORG': [], 'PER': []},
  'scores': {'PER': (1.0, 1.0, 1.0),
   'ORG': (1.0, 1.0, 1.0),
   'LOC': (0.0, 0, 0),
   'MISC': (0, 0.0, 0)}}}

In [26]:
def calculate_average_metrics(data):
    model_type = 'ft_model' #'base_model'
    
    # Initialize variables to store sum of precision, recall, and f1_score for each entity type
    sum_precision = {'PER': 0, 'ORG': 0, 'LOC': 0, 'MISC': 0}
    sum_recall = {'PER': 0, 'ORG': 0, 'LOC': 0, 'MISC': 0}
    sum_f1_score = {'PER': 0, 'ORG': 0, 'LOC': 0, 'MISC': 0}
    num_instances = len(data)

    # Iterate over the list of dictionaries
    for entry in data:
        metrics = entry[model_type]['scores']
        for entity_type, (precision, recall, f1_score) in metrics.items():
            sum_precision[entity_type] += precision
            sum_recall[entity_type] += recall
            sum_f1_score[entity_type] += f1_score

    # Calculate average precision, recall, and f1_score for each entity type
    avg_precision = {entity_type: sum_precision[entity_type] / num_instances for entity_type in sum_precision}
    avg_recall = {entity_type: sum_recall[entity_type] / num_instances for entity_type in sum_recall}
    avg_f1_score = {entity_type: sum_f1_score[entity_type] / num_instances for entity_type in sum_f1_score}

    return avg_precision, avg_recall, avg_f1_score


In [54]:
# base_model
# Calculate average precision, recall, and f1_score
avg_precision, avg_recall, avg_f1_score = calculate_average_metrics(responses)

# Print the results
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-score:", avg_f1_score)


Average Precision: {'PER': 0.7777777777777778, 'ORG': 0.6666666666666666, 'LOC': 0.7777777777777778, 'MISC': 0.6666666666666666}
Average Recall: {'PER': 0.7777777777777778, 'ORG': 0.6666666666666666, 'LOC': 0.7777777777777778, 'MISC': 0.6666666666666666}
Average F1-score: {'PER': 0.7777777777777778, 'ORG': 0.6666666666666666, 'LOC': 0.7777777777777778, 'MISC': 0.6666666666666666}


In [27]:
# ft_model
# Calculate average precision, recall, and f1_score
avg_precision, avg_recall, avg_f1_score = calculate_average_metrics(responses)

# Print the results
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)
print("Average F1-score:", avg_f1_score)

Average Precision: {'PER': 1.0, 'ORG': 1.0, 'LOC': 0.5, 'MISC': 0.6}
Average Recall: {'PER': 0.9666666666666666, 'ORG': 0.9, 'LOC': 0.45, 'MISC': 0.6}
Average F1-score: {'PER': 0.9800000000000001, 'ORG': 0.9333333333333332, 'LOC': 0.4666666666666667, 'MISC': 0.6}


In [57]:
def calculate_average_for_each_position(data):
    # Initialize variables to store sum of values for each position
    sum_position1 = 0
    sum_position2 = 0
    sum_position3 = 0
    num_instances = len(data)

    # Iterate over the list of tuples
    for tpl in data:
        sum_position1 += tpl[0]
        sum_position2 += tpl[1]
        sum_position3 += tpl[2]

    # Calculate average for each position
    avg_position1 = sum_position1 / num_instances
    avg_position2 = sum_position2 / num_instances
    avg_position3 = sum_position3 / num_instances

    return round(avg_position1,2), round(avg_position2,2), round(avg_position3,2)



In [58]:
#percision, recall, f1
calculate_average_for_each_position([x['base_model']['scores']['PER'] for x in responses])

(0.78, 0.78, 0.78)

In [59]:
calculate_average_for_each_position([x['base_model']['scores']['ORG'] for x in responses])

(0.67, 0.67, 0.67)

In [60]:
calculate_average_for_each_position([x['base_model']['scores']['LOC'] for x in responses])

(0.78, 0.78, 0.78)

In [61]:
calculate_average_for_each_position([x['base_model']['scores']['MISC'] for x in responses])

(0.67, 0.67, 0.67)